<a href="https://colab.research.google.com/github/LToni51/LToni51/blob/main/integracaosparkepandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

UTILIZAÃO DO PYSPARK

In [1]:
pip install pyspark

f

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
print(spark)

In [4]:
dataset = spark.read.csv('/content/sample_data/california_housing_test.csv', header=True, inferSchema=True )

EXIBINDO O DATASET

In [6]:
dataset.head()

Row(longitude=-122.05, latitude=37.37, housing_median_age=27.0, total_rooms=3885.0, total_bedrooms=661.0, population=1537.0, households=606.0, median_income=6.6085, median_house_value=344700.0)

CONTANDO AS LINHAS DO DATASET

In [7]:
dataset.count()

3000

Agora vamos criar uma tabela SQL temporária com os dados do “dataset”. Para isso, devemos executar o código abaixo:

In [9]:
dataset.createOrReplaceTempView('tabela_temporaria')
print(spark.catalog.listTables())

[Table(name='tabela_temporaria', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]


fazendo consulta sql

In [10]:
query ='FROM tabela_temporaria SELECT longitude, latitude LIMIT 5'
saida = spark.sql(query)
saida.show()

+---------+--------+
|longitude|latitude|
+---------+--------+
|  -122.05|   37.37|
|   -118.3|   34.26|
|  -117.81|   33.78|
|  -118.36|   33.82|
|  -119.67|   36.33|
+---------+--------+



A primeira parte da nossa solução vai ser dividida nos seguintes passos:


Implementar a consulta SQL na tabela “tabela_temporaria” que já carregamos no Spark para retornar a quantidade máxima de quartos.
Executar a consulta SQL no Spark e, assim, obter um DataFrame do Spark.
Converter o resultado da etapa anterior para um DataFrame do Pandas.
Imprimir o resultado da consulta.
Converter o valor do DataFrame para um valor inteiro.

In [15]:
query1 = 'SELECT MAX(total_rooms) as maximo_quartos FROM tabela_temporaria'
q_maximo_quartos = spark.sql(query1)
pd_maximo_quartos = q_maximo_quartos.toPandas()
print('a quantidade maxima de quartos é {}'.format(pd_maximo_quartos['maximo_quartos']))
qtd_maximo_quartos = int(pd_maximo_quartos.loc[0, 'maximo_quartos'])

a quantidade maxima de quartos é 0    30450.0
Name: maximo_quartos, dtype: float64


In [13]:
dataset.printSchema()

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)

